In [13]:
#Code proposé par M.Nataf:Séance 1
import scipy.io as spi
import numpy as np
import matplotlib.pyplot as plt

In [14]:
mat=spi.loadmat("mnist-original.mat")
data=np.transpose(mat['data'])
label=np.array(mat['label']) #label: chiffre numérisé
label=label.astype(int) #Les labels sont stockés en flottants, on les convertit en entiers

NOUS MODIFIONS PRINCIPALEMENT NOTRE METHODE DE DEFINITION DE LA BASE DE TEST 

Découpe aléatoire de la base de données en base d'apprentissage et base de test (fonction modifiée)

In [15]:
Y,y=data,label[0]
#Ona modifié cette fonction afin d'obtenir une base d'apprentissage correspondant à 80% de la base de données (comme avant)
# mais cette fois les 20% de données utilisées pour tester le modèle sont tirés de la base d'apprentissage également
def decoupe_aleatoire_base(Y,y):
    m=np.random.permutation((len(y))) #permutation arbitraire
    Y_m=Y[m] #data après permutation
    y_m=y[m] #labels après permutation
    n=len(Y) #nombre d'images dans la base donnée
    n_20=20*n/100
    n_20=int(n_20)
    n_80=80*n/100 #nombre d'images dans la base d'apprentissage
    n_80=int(n_80)
    data_app=Y_m[:n_80] #base d'apprentissage
    label_app=y_m[:n_80]
    data_test=Y_m[:n_20] #base de tests définie sur une portion de la base d'apprentissage cette fois 
    label_test=y_m[:n_20]
    return data_app,label_app,data_test,label_test

Dans la base d'apprentissage, calcul des centroides des chiffres

In [16]:
def calcul_centroides(data_app,label_app):
    X,x=data_app,label_app 
    moy_chiff=[]
    for i in range(10):
        moy=np.mean(X[x==i],axis=0)
        moy_chiff+=[moy]
    return moy_chiff

Méthode d'estimation par rapport à la distance euclidienne

In [17]:
#Définition de la fonction qui estime le chiffre d'un vecteur de la base de tests:
def estim_chiffre_1(v,moy_chiff):
    distances=np.array([np.linalg.norm(v-u) for u in moy_chiff]) #distances entre v et les "chiffres moyens"
    return np.argmin(distances)

Méthode d'estimation par rapport à la "distance" cosine

In [18]:
#On définit la fonction cosine:
def cosine(u,v):
    return np.inner(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

In [19]:
#Fonction qui estime le chiffre d'un vecteur de la base de tests vis a vis de la "distance" cosine:
def estim_chiffre_2(v,moy_chiff):
    distances=np.array([cosine(u,v) for u in moy_chiff]) #distances cosine entre v et les "chiffres moyens"
    return np.argmax(distances) #On prend le max car plus l'angle est petit, plus le cos est grand

Méthode d'estimation par rapport à la norme p

In [20]:
#Fonction qui estime le chiffre d'un vecteur de la base de tests vis a vis de la norme p:
def estim_chiffre_p(p,v,moy_chiff):
    distances=np.array([np.linalg.norm(v-u,p) for u in moy_chiff]) #distances entre v et les "chiffres moyens" en norme p
    return np.argmin(distances)

Calcul du pourcentage de prédictions correctes sur la base de tests

In [21]:
#Labels estimés pour les vecteurs de la base de tests et pour une certaine distance:
def precision_estimation_label(data_test,label_test,methode,p,moy_chiff): # renseigner un p quelconque si on n'utilise pas la norme p (par ex p=0)
    k=len(data_test)
    label_estim=np.zeros(k)
        
    if (methode != estim_chiffre_p):      
        for i in range(k):
            label_estim[i]=methode(data_test[i],moy_chiff)
            #print("la précision obtenue est de", np.mean(label_estim==label_test))
            
    else:
        for i in range(k):
            label_estim[i]=methode(p, data_test[i],moy_chiff)
            #print("la précision obtenue pour p={} est de".format(p), np.mean(label_estim==label_test))
        
    return np.mean(label_estim==label_test) #  pourcentage de prédictions correctes sur la base de tests.

Fonction permettant de calculer la précision d une méthode (pour une seule découpe initiale)

In [22]:
def Precision_methode(methode,p,Y,y):    
    data_app,label_app,data_test,label_test=decoupe_aleatoire_base(Y,y)
    moy_chiff=calcul_centroides(data_app,label_app)
    precision=precision_estimation_label(data_test,label_test,methode,p,moy_chiff)
    return precision

VOYONS SI NOTRE MODELE SOUFFRE D' "OVERFIT"

Méthode norme euclidienne

In [23]:
Precision_methode(estim_chiffre_1,0,Y,y)

0.81

(Pour rappel, nous avions trouvé 80.9% de précision moyenne pour cette méthode, avec 100 découpes initiales différentes, et une base de test scindée de la base d'apprentissage.)

Méthode "distance" cosine

In [24]:
Precision_methode(estim_chiffre_2,0,Y,y)

0.8202142857142857

(Pour rappel, nous avions trouvé 81.5% de précision moyenne pour cette méthode, avec 100 découpes initiales différentes,
et une base de test scindée de la base d'apprentissage.)

Méthode norme p

In [25]:
for p in range(3,11):
    print("précision pour", p ,"est de", Precision_methode(estim_chiffre_p,p,Y,y))

précision pour 3 est de 0.8214285714285714
précision pour 4 est de 0.8067857142857143
précision pour 5 est de 0.8031428571428572
précision pour 6 est de 0.7869285714285714
précision pour 7 est de 0.7822857142857143
précision pour 8 est de 0.7750714285714285
précision pour 9 est de 0.7659285714285714
précision pour 10 est de 0.7694285714285715


(Pour rappel, pour une seule découpe initiale, avec une base de test scindée de la base d'apprentissage,
nous avions trouvé en tant que précisions moyennes:
 
précision pour 3 est de 0.8205

précision pour 4 est de 0.815

précision pour 5 est de 0.8049285714285714

précision pour 6 est de 0.7945714285714286

précision pour 7 est de 0.786

précision pour 8 est de 0.7793571428571429

précision pour 9 est de 0.7722142857142857

précision pour 10 est de 0.7650714285714286)

CONCLUSION

Les estimations de précision obtenues en prenant une base de test au sein de la base d'apprentissage et non pas au sein d'une base scindée donnent toutes à moins de 2% de différence près les mêmes résultats que nos premières estimations avec deux bases scindées. 
Cela semble indiquer que notre modèle ne souffre pas d'overfit' (qui aurait généré des précisions bien meilleures dans le cas de bases non scindées par rapport au cas de bases scindées).